In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

c:\Users\owen_\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_json("database_1000.json")

In [26]:
df["Description"] = df["Description"].astype(str)

In [27]:
Descriptions = df["Description"].tolist()

In [28]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [29]:
embeddings = model.encode(Descriptions, show_progress_bar=True)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
#num_clusters = 1000
#kmeans = KMeans(n_clusters=num_clusters, random_state=42)
#kmeans.fit(embeddings)
#df["cluster"] = kmeans.labels_

c:\Users\Jonat\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
c:\Users\Jonat\anaconda3\Lib\site-packages\sklearn\base.py:1473: ConvergenceWarning: Number of distinct clusters (999) found smaller than n_clusters (1000). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [103]:
def recommend_from_prompt(prompt, top_n=20):
    # Encode the input prompt
    prompt_embedding = model.encode([prompt])

    # Predict which cluster it belongs to
    #cluster_id = kmeans.predict(prompt_embedding)[0]

    # Get reviews in that cluster
    #cluster_df = df[df["cluster"] == cluster_id].copy()

    # Compute cosine similarity between the prompt and all reviews in the cluster
    #cluster_embeddings = model.encode(cluster_df["Description"].tolist())
    similarities = cosine_similarity(prompt_embedding, embeddings)[0]

    # Add similarity scores and sort
    df["similarity"] = similarities
    top_reviews = df.sort_values(by="similarity", ascending=False).head(top_n)

    return top_reviews[["Description", "similarity","Name"]]

In [104]:
prompt = "Worker placement games"  
recommended = recommend_from_prompt(prompt)

for idx, row in recommended.iterrows():
    print(f"Name: {row['Name']} | Score: {row['similarity']:.4f}")

Name: Dice Hospital | Score: 0.6156
Name: Kitchen Rush | Score: 0.5015
Name: Fantastic Factories | Score: 0.4886
Name: Coal Baron | Score: 0.4723
Name: Chinatown | Score: 0.4584
Name: Ora et Labora | Score: 0.4347
Name: Cacao | Score: 0.4285
Name: Maquis | Score: 0.4265
Name: Egizia | Score: 0.4206
Name: Bruxelles 1893 | Score: 0.4174
Name: The Pursuit of Happiness | Score: 0.4149
Name: Quadropolis | Score: 0.4143
Name: Food Chain Magnate | Score: 0.4143
Name: Everdell | Score: 0.4142
Name: Belfort | Score: 0.4075
Name: Vasco da Gama | Score: 0.3978
Name: Hansa Teutonica | Score: 0.3970
Name: Spyrium | Score: 0.3910
Name: Navegador | Score: 0.3899
Name: The Princes of Florence | Score: 0.3897


In [75]:
df[df['Name']=='Near and Far']

,Name,Rank,Link,Geek Rating,Average ratings,Voters,min_players,max_players,min_duration,max_duration,weight,mechanics,Description,cluster
243,Near and Far,244,https://boardgamegeek.com/boardgame/195421/nea...,7.273,7.67,10549,2,4,90,120,2.89,"[Closed Drafting, Contracts, Dice Rolling, End...","Four wanderers search for the Last Ruin, a cit...",12


In [42]:
df[df['cluster']==8]

,Name,Rank,Link,Geek Rating,Average ratings,Voters,min_players,max_players,min_duration,max_duration,weight,mechanics,Description,cluster
14,Great Western Trail,15,https://boardgamegeek.com/boardgame/193738/gre...,8.018,8.19,41433,2,4,75,150,3.70,"[Deck, Bag, and Pool Building, Hand Management...",America in the 19th century: You are a rancher...,8
25,Great Western Trail: Second Edition,26,https://boardgamegeek.com/boardgame/341169/gre...,7.906,8.32,13620,1,4,75,150,3.72,"[Deck, Bag, and Pool Building, Hand Management...",America in the 19th century: You are a rancher...,8
27,Lost Ruins of Arnak,28,https://boardgamegeek.com/boardgame/312484/los...,7.895,8.07,46148,1,4,30,120,2.93,"[Contracts, Deck, Bag, and Pool Building, Mult...","On an uninhabited island in uncharted seas, ex...",8
29,Terra Mystica,30,https://boardgamegeek.com/boardgame/120677/ter...,7.889,8.06,48524,2,5,60,150,3.97,"[Chaining, End Game Bonuses, Hexagon Grid, Inc...",In the land of Terra Mystica dwell 14 differen...,8
36,Everdell,37,https://boardgamegeek.com/boardgame/199792/eve...,7.841,8.02,56981,1,4,40,80,2.82,"[Contracts, End Game Bonuses, Hand Management,...","Within the charming valley of Everdell, beneat...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,Santiago,894,https://boardgamegeek.com/boardgame/8125/santiago,6.684,7.16,5431,3,5,75,na,2.48,"[Area Majority / Influence, Auction / Bidding,...",Santiago is about cultivating and watering fie...,8
914,Morels,915,https://boardgamegeek.com/boardgame/122298/morels,6.671,6.99,9447,2,na,30,na,1.74,"[Hand Management, Open Drafting, Set Collection]","The woods are old-growth, dappled with sunligh...",8
972,Akrotiri,973,https://boardgamegeek.com/boardgame/154458/akr...,6.634,7.17,4741,2,na,45,na,2.64,"[Action Points, Hand Management, Map Addition,...",Akrotiri places you in the role of an explorer...,8
980,Palm Island,981,https://boardgamegeek.com/boardgame/239464/pal...,6.630,7.08,6246,1,2,15,na,1.75,"[Cooperative Game, Deck, Bag, and Pool Buildin...",Palm Island is a portable game that you can ta...,8


In [41]:
df[df['Name']=='Agricola']

,Name,Rank,Link,Geek Rating,Average ratings,Voters,min_players,max_players,min_duration,max_duration,weight,mechanics,Description,cluster
53,Agricola,54,https://boardgamegeek.com/boardgame/31260/agri...,7.744,7.87,73445,1,5,30,150,3.64,"[Automatic Resource Growth, Closed Drafting, E...",Description originally from BoardgameNews In A...,8
